In [0]:
import urllib.parse
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from google.colab import drive
import time
drive.mount('/content/drive')

In [0]:
figure_folder = '/content/drive/My Drive/pytorch_project/figures/'
base_review_url_us = 'https://myanimelist.net/anime/'

In [0]:
def getbsObj(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return(None)
    try:
        bsObj = BeautifulSoup(html.read())
    except AttributeError as e:
        return(None)
    return(bsObj)

In [0]:
def get_data(index):
  review_url = base_review_url_us + str(index)
  bsObj = getbsObj(review_url)
  #date = bsObj.find(text= 'Premiered:').parent.parent.get_text()[-5:-1]
  name = bsObj.findAll("span", {"itemprop":"name"})[0].get_text()
  if name == None:
    return None
  try:
    score = float(bsObj.findAll("div", {"class":"fl-l score"})[0].get_text())
  except:
    score = 'NA'
  rating = bsObj.find(text= 'Rating:').parent.parent.get_text()[11:-3]
  #Producer_US = bsObj.find(text= 'Studios:').parent.parent.get_text()[10:-1]
  genre = bsObj.find(text= 'Genres:')
  figure_url = bsObj.findAll("img", class_='ac')[0].get('src')
  genres = []
  for genre in genre.parent.next_siblings:
    genres.append(genre.string)
  urllib.request.urlretrieve(figure_url, figure_folder + str(index) + '.jpg')
  print('success -- ', str(index))
  return [index, name, score, rating, genres]

In [0]:
data = []
for index in range(10601, 20001):
  try:
    r = get_data(index)
    if r is not None:
      data.append(r)
  except:
    print('fail -- ', str(index) + '--' + str(len(data)))
    pass
  if index % 200 == 0:
    df = pd.DataFrame(data, columns = ['index','name','score','rating','genres'])
    df['genres'] = df['genres'].apply(lambda x: [xx for xx in x if xx not in ['\n',', ']])
    df.to_csv('/content/drive/My Drive/anime_data_' + str(index - 200) + '_' + str(index) + '.csv', index = False)
    data = []
    time.sleep(30)